In [2]:
%%capture
%pip install accelerate peft transformers

In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 50.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.3.17 requires tyro, which is not installed.
unsloth-zoo 2025.3.17 requires protobuf<4.0.0, but you have protobuf 5.29.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 8.2 MB/s eta 0:00:00


In [ ]:
import os
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
import torch
from unsloth import FastLanguageModel

In [ ]:
from huggingface_hub import HfApi, login

# Авторизация
login(token=hf_token)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path="unsloth/Meta-Llama-3.1-8B")

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = 8096,
    dtype = None,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import json
import pandas as pd

In [ ]:
with open("train_df", "r", encoding="utf-8") as f:
  df_train_json = json.load(f)

In [ ]:
df = pd.DataFrame.from_dict(df_train_json)
df

,instruction,input,output
0,Ты - юридический консультант. Тебе даётся след...,Положен ли мне штраф с страховой компании за о...,"Да, согласно п. 3 ст. 16.1 Закона об ОСАГО, пр..."
1,Ты - юридический консультант. Тебе даётся след...,Могу ли я требовать доплаты от страховой компа...,"Да, согласно п. 1 ст. 929 ГК РФ и ст. 12 Федер..."
2,Ты - юридический консультант. Тебе даётся след...,"Можно ли взыскать с виновника ДТП разницу, есл...","Да, по ст. 1072 ГК РФ и ст. 1064 ГК РФ, виновн..."
3,Ты - юридический консультант. Тебе даётся след...,Обязан ли водитель (арендатор) возмещать ущерб...,"Нет, если водитель управлял ТС на основании пу..."
4,Ты - юридический консультант. Тебе даётся след...,"Можно ли взыскать ущерб с ООО «Импульс», если ...","Да, поскольку по ст. 1079 ГК РФ владелец источ..."
...,...,...,...
11920,Ты - юридический консультант. Тебе даётся след...,Могу ли я требовать компенсацию морального вре...,"Да, согласно ст. 151, 1100 ГК РФ, компенсация ..."
11921,Ты - юридический консультант. Тебе даётся след...,Кто должен возмещать вред — водитель или предп...,"По ст. 1068 и 1079 ГК РФ, ответственность несе..."
11922,Ты - юридический консультант. Тебе даётся след...,Может ли суд повторно привлечь водителя по ч. ...,"Да, если при новом рассмотрении суд установит,..."
11923,Ты - юридический консультант. Тебе даётся след...,"Могу ли я обжаловать постановление суда, если ...","Да, согласно ч. 3 ст. 25.2 КоАП РФ, дело должн..."


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.05, shuffle=True)

In [ ]:
len(train_df), len(test_df)

(11328, 597)

In [ ]:
val_df, test_df = train_test_split(test_df, test_size=0.20, shuffle=True)

In [ ]:
len(val_df), len(test_df)

(477, 120)

In [ ]:
test_df_pandas = test_df

In [ ]:
pd.to_csv("train_df")
pd.to_csv("val_df")
pd.to_csv("test_df")

In [ ]:
train_df.instruction[0]

<ipython-input-35-db0452b8464c>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  train_df.instruction[0]


'Ты - юридический консультант. Тебе даётся следующий контекст: Гражданин Республики Киргизия находился на территории Российской Федерации без документов, подтверждающих право на пребывание свыше 90 суток, что является нарушением режима пребывания. Судья Апшеронского районного суда признал его виновным по ч. 1.1 ст. 18.8 КоАП РФ и назначил штраф 5 000 рублей с административным выдворением. Заявитель обжаловал решение, считая его необоснованным, особенно в части выдворения. Исходя из этого, ответь на вопрос: '

In [ ]:
prompt_format = """Ниже приведена инструкция, описывающая задачу, в сочетании с вводом, обеспечивающим дополнительный контекст. Напишите ответ, который соответствующим образом завершает запрос.

### Инструкция:
{}

### Ввод:
{}

### Ответ:
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt_format.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
train_df = Dataset.from_pandas(train_df)
val_df = Dataset.from_pandas(val_df)

In [ ]:
test_df = Dataset.from_pandas(test_df)

In [ ]:
train_df = train_df.map(formatting_prompts_func, batched = True)
val_df = val_df.map(formatting_prompts_func, batched = True)
test_df = test_df.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/11328 [00:00<?, ? examples/s]

Map:   0%|          | 0/477 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

In [ ]:
train_df['text'][0]

'Ниже приведена инструкция, описывающая задачу, в сочетании с вводом, обеспечивающим дополнительный контекст. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nТы - юридический консультант. Тебе даётся следующий контекст: Гражданин Республики Киргизия находился на территории Российской Федерации без документов, подтверждающих право на пребывание свыше 90 суток, что является нарушением режима пребывания. Судья Апшеронского районного суда признал его виновным по ч. 1.1 ст. 18.8 КоАП РФ и назначил штраф 5 000 рублей с административным выдворением. Заявитель обжаловал решение, считая его необоснованным, особенно в части выдворения. Исходя из этого, ответь на вопрос: \n\n### Ввод:\nМогут ли отменить административное выдворение, если я признаю вину и заплачу штраф?\n\n### Ответ:\nНет, ч. 1.1 ст. 18.8 КоАП РФ предусматривает безальтернативное наказание в виде штрафа с выдворением. Даже при признании вины и уплате штрафа выдворение обязательно, так как это п

In [ ]:
import wandb
wandb.login()

In [ ]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

BATCH_SIZE = 64
GRADIENT_ACCUMULATION_STEPS = 2
LEARNING_RATE = 2e-4
TRAIN_STEPS = 500
# TRAIN_EPOCHS = 1
OUTPUT_DIR = "/outputs"

training_arguments = TrainningArguments(
    per_device_train_batch_size = BATCH_SIZE,
    gradient_accumulation_steps = GRADIENT_ACCUMULATION_STEPS,
    warmup_steps = 50,
    # num_train_epochs = 1, # Set this for 1 full training run.
    max_steps = TRAIN_STEPS,
    learning_rate = LEARNING_RATE,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 10,
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 10,
    save_steps = 10,
    save_total_limit = 3,
    load_best_model = True,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = OUTPUT_DIR,
    report_to = "wandb" #"tensorboard", # Use this for WandB etc
    # overwrite_output_dir=True
)

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_df,
    eval_dataset = val_df,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_arguments,
)

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 11,328 | Num Epochs = 1 | Total steps = 1,416
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.790000
2,1.695000
3,1.617000
4,1.599300
5,1.542100
6,1.470800
7,1.277100
8,1.162000
9,1.162800
10,1.062000


In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")